<a href="https://colab.research.google.com/github/nisanuro/CNG562-Assignment-2/blob/naive/CNG562_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn import metrics, datasets, preprocessing
%matplotlib inline
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB, ComplementNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [0]:
def randomOneHoldout(X_train, Y_train):

  x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=0)

  return x_train, x_test, y_train, y_test

In [0]:
def stratifiedOneHoldout(X_train, Y_train):
  
  x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=0, stratify=Y_train)
  
  return x_train, x_test, y_train, y_test

In [0]:
def NaiveBayes(X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=9)
    '''
    # Gaussian Naive Bayes
    gaussian = GaussianNB()
    NaiveBayesValidations(X_train, Y_train, gaussian)

    y_pred = gaussian.predict(X_test)
    print("Accuracy - unseen data: ", metrics.accuracy_score(Y_test, y_pred)*100)
    
    fourError(X, Y, gaussian)
    
    
    # Multinomial Naive Bayes
    multinomial = MultinomialNB(fit_prior=True)
    NaiveBayesValidations(X_train, Y_train, multinomial)

    y_pred = multinomial.predict(X_test)
    print("Accuracy - unseen data: ", metrics.accuracy_score(Y_test, y_pred)*100)

    fourError(X, Y, multinomial)
    
    
    # Bernoulli Naive Bayes
    
    bernoulli = BernoulliNB(binarize = 1.5)
    bernoulli.fit(X_train, Y_train)
    #NaiveBayesValidations(X_train, Y_train, bernoulli)

    
    y_pred = bernoulli.predict(X_test)
    print("Accuracy - unseen data: ", metrics.accuracy_score(Y_test, y_pred)*100)

    fourError(X, Y, bernoulli)
    
    #BernoulliBinarize(X_train, Y_train)
    
   
    # Complement Naive Bayes
    complement = ComplementNB()
    NaiveBayesValidations(X_train, Y_train, complement)

    y_pred = complement.predict(X_test)
    print("Accuracy - unseen data: ", metrics.accuracy_score(Y_test, y_pred)*100)

    fourError(X, Y, complement)
    
    #complementTuning(X_train, Y_train)
    '''

    # Categorical Naive Bayes
    categorical = CategoricalNB()
    NaiveBayesValidations(X_train, Y_train, categorical)                

    y_pred = categorical.predict(X_test)
    print("Accuracy - unseen data: ", metrics.accuracy_score(Y_test, y_pred)*100)

    fourError(X, Y, categorical)  

In [0]:
def complementTuning(X_train, Y_train):
    fit_prior = [True, False]
    norm = [True, False]

    for i in fit_prior:
        for j in norm:
            model = ComplementNB(fit_prior = i, norm = j)

            cv = cross_val_score(model, X_train, Y_train, cv=10, scoring='accuracy')

            print("fit_prior = ", i, "   norm = ", j, "   Accuracy: ", cv.mean()*100)

In [0]:
def fourError(X, Y, model):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=15, stratify=Y)
    
    Train_x, TrainDev_x, Train_y, TrainDev_y = train_test_split(X_train, Y_train, test_size=0.2, random_state=0, stratify=Y_train)
    Dev_x, Test_x, Dev_y, Test_y = train_test_split(X_test, Y_test, test_size=0.5, random_state=15, stratify=Y_test)

    model.fit(Train_x, Train_y)

    trainDev_pred = model.predict(TrainDev_x)
  
    print("Train-Train Dev,   e1:", metrics.mean_squared_error(TrainDev_y, trainDev_pred))
    print("Decision Tree Accuracy: ", 1 - metrics.mean_squared_error(TrainDev_y, trainDev_pred),"\n")

    dev_pred = model.predict(Dev_x)

    print("Train-Dev,   e2", metrics.mean_squared_error(Dev_y, dev_pred))
    print("Decision Tree Accuracy: ", 1 - metrics.mean_squared_error(Dev_y, dev_pred),"\n")

    test_pred = model.predict(Test_x)

    print("Train-Test,   e3: ", metrics.mean_squared_error(Test_y, test_pred))
    print("Decision Tree Accuracy: ", 1 - metrics.mean_squared_error(Test_y, test_pred),"\n")

    devTest_pred = model.predict(X_test)

    print("Train-(Dev+Test),   e4: ", metrics.mean_squared_error(Y_test, devTest_pred))
    print("Decision Tree Accuracy: ", 1 - metrics.mean_squared_error(Y_test, devTest_pred),"\n")

In [0]:
def BernoulliBinarize(X_train, Y_train):
    binarize = [0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3]

    for i in binarize:

        model = BernoulliNB(binarize = i)
        cv = cross_val_score(model, X_train, Y_train, cv=5, scoring='accuracy')
        print("Binarize = ", i, "    Accuracy: ", cv.mean()*100)

        

In [0]:
def NaiveBayesValidations(X_train, Y_train, model):
    print("\n" + str(model).split('N')[0] + " Naive Bayes Accuracy\n")
    # 5-Fold
    cv = cross_val_score(model, X_train, Y_train, cv=5, scoring='accuracy')

    print("5-Fold: ", cv.mean()*100)
    
    # 10-Fold
    cv = cross_val_score(model, X_train, Y_train, cv=10, scoring='accuracy')

    print("10-Fold: ", cv.mean()*100)

    # Random One Holdout
    x_train, x_test, y_train, y_test = randomOneHoldout(X_train, Y_train)
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    print("Random One Holdout: ", metrics.accuracy_score(y_test, y_pred)*100)

    # Stratified One Holdout
    x_train, x_test, y_train, y_test = stratifiedOneHoldout(X_train, Y_train)

    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    print("Stratified One Holdout: ", metrics.accuracy_score(y_test, y_pred)*100)



In [0]:
def DecisionTree(X, Y):
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
    
    # 5-Fold
    print("\n5-Fold: ")
    tuningDepth(X_train, Y_train, 0)

    # 10-Fold
    print("\n10-Fold: ")
    tuningDepth(X_train, Y_train, 1)

    # Random One Holdout
    print("\nRandom One Holdout: ")   
    tuningDepth(X_train, Y_train, 2)
  
    # Stratified One Holdout
    print("\nStratified One Holdout: ")
    tuningDepth(X_train, Y_train, 3)
    
    #
    #   Continue with 10-Fold, Depth = 3
    #
    
    print("5-Fold, Depth=5\n")
    tuningSplit(X_train, Y_train)
    
    #
    #   Continue with criterion = 'gini', splitter = 'best', min_samples_split = 2
    #                     all of them are default values
    print("5-Fold, depth = 5, criterion = 'gini', splitter = 'best, min_samples_split = 2\n")
    tuningClassWeight(X_train, Y_train)

    #
    #   Continue with class_weight = None,   default
    #
    
    clf = DecisionTreeClassifier(max_depth = 3)
    '''
    clf.fit(X_train, Y_train)
    
    y_pred = clf.predict(X_test)
    print("Accuracy: ", metrics.accuracy_score(Y_test, y_pred)*100)
    '''
    fourError(X, Y, clf)

In [0]:
def tuningClassWeight(X_train, Y_train):
    # No class weight
    clf = DecisionTreeClassifier(max_depth = 3, random_state = 0)
    clf.fit(X_train, Y_train)

    cv = cross_val_score(clf, X_train, Y_train, cv=10, scoring='accuracy')
    print("Class weight: None           Accuracy: ", cv.mean()*100)

    # Balanced class weight
    clf = DecisionTreeClassifier(max_depth = 3, random_state = 0, class_weight ='balanced')
    clf.fit(X_train, Y_train)

    cv = cross_val_score(clf, X_train, Y_train, cv=10, scoring='accuracy')
    print("Class weight: Balanced       Accuracy: ", cv.mean()*100)

In [0]:
def tuningSplit(X_train, Y_train):
    criterion = ["gini", "entropy"]
    splitter = ["best", "random"]

    for i in criterion:
        for j in splitter:
            clf = DecisionTreeClassifier(criterion = i, splitter = j, max_depth = 3, random_state = 0)
            clf.fit(X_train, Y_train)

            cv = cross_val_score(clf, X_train, Y_train, cv=10, scoring='accuracy')
            print("Criterion: ", i, "   Splitter: ", j, "   Accuracy: ", cv.mean()*100)

    for i in range(2, 10):
        clf = DecisionTreeClassifier(max_depth = 3, min_samples_split = i, random_state = 0)
        clf.fit(X_train, Y_train)

        cv = cross_val_score(clf, X_train, Y_train, cv=10, scoring='accuracy')
        print("min_samples_split: ", i, "   Accuracy: ", cv.mean()*100)

In [0]:
def tuningDepth(X_train, Y_train, val):

    max_depth_range = list(range(1, 10))
    max_depth_range.append(str("None"))
    

    for depth in max_depth_range:
        if (val == 0):
            if(depth == "None"):
                clf = DecisionTreeClassifier(random_state = 0)
                clf.fit(X_train, Y_train)
            else:               
                clf = DecisionTreeClassifier(max_depth = depth, random_state = 0)
                clf.fit(X_train, Y_train)

            cv = cross_val_score(clf, X_train, Y_train, cv=5, scoring='accuracy')
            accuracy = cv.mean()*100
            print("Depth: ", depth, " Accuracy: ", accuracy)
            
        elif (val == 1):
            if(depth == "None"):
                clf = DecisionTreeClassifier(random_state = 0)
                clf.fit(X_train, Y_train)
            else:  
                clf = DecisionTreeClassifier(max_depth = depth, random_state = 0)
                clf.fit(X_train, Y_train)

            cv = cross_val_score(clf, X_train, Y_train, cv=10, scoring='accuracy')
            accuracy = cv.mean()*100
            print("Depth: ", depth, " Accuracy: ",accuracy)
            
        elif(val == 2):            
            x_train, x_test, y_train, y_test = randomOneHoldout(X_train, Y_train)

            if(depth == "None"):
                clf = DecisionTreeClassifier(random_state = 0)
                clf.fit(X_train, Y_train)
            else:  
                clf = DecisionTreeClassifier(max_depth = depth, random_state = 0)
                clf.fit(x_train, y_train)

            accuracy = clf.score(x_test, y_test)*100
            print("Depth: ", depth, " Accuracy: ", accuracy)
            
        elif(val == 3):
            x_train, x_test, y_train, y_test = stratifiedOneHoldout(X_train, Y_train)

            if(depth == "None"):
                clf = DecisionTreeClassifier(random_state = 0)
                clf.fit(X_train, Y_train)
            else:  
                clf = DecisionTreeClassifier(max_depth = depth, random_state = 0)
                clf.fit(x_train, y_train)

            accuracy = clf.score(x_test, y_test)*100
            print("Depth: ", depth, " Accuracy: ", accuracy)
            
        else:
            print("Invalid validation tech.")



In [0]:
def displayAccuracy(X, Y):
    
    NaiveBayes(X, Y)
    #DecisionTree(X, Y)

In [0]:
def zValues(df):
    
    cols = list(df.columns)
    cols.remove('Index')

    for col in cols:
        col_zscore = col + '_zscore'
        df[col_zscore] = (df[col] - df[col].mean())/df[col].std(ddof=0)

    return df

In [0]:
def outliers(df):  
    return df.loc[(df._1_zscore > 2.5) | (df._2_zscore > 2.5) | (df._2_zscore > 2.5) | (df._3_zscore > 2.5)]

In [0]:
def subDatasets(df):
    target0 = []
    target1 = []
    target2 = []
    for row in df.itertuples():
        if row.target == 0:
            target0.append(row)
            target0_df = pd.DataFrame(target0)
        elif row.target == 1:
            target1.append(row)
            target1_df = pd.DataFrame(target1)
        else:
            target2.append(row)    
            target2_df = pd.DataFrame(target2)  

    dfs = [target0_df, target1_df, target2_df]  

    for df in dfs:
        df.drop(columns=['target'])
    
    return target0_df, target1_df, target2_df

In [0]:
if __name__ == '__main__':

  iris = datasets.load_iris()
  
  X = iris.data
  Y = iris.target
  
  # L1 normalization
  l1_norm = preprocessing.normalize(X, norm="l1")
  # Mean removal
  mean_removal = preprocessing.scale(X)

  #Displaying result according to each type of methods and regression model
  #print("\nRaw: ")
  #displayAccuracy(X,Y)
  '''
  print("\nL1 Normalization: ")
  displayAccuracy(l1_norm,Y)
  
  print("\nMean Removal: ")
  displayAccuracy(mean_removal,Y)'''

  df = pd.DataFrame(iris.data, columns=iris.feature_names)
  df['target'] = iris.target
  pd.set_option('display.max_rows', df.shape[0]+1)
 
  t0, t1, t2 = subDatasets(df)
  
  z0 = zValues(t0)
  z1 = zValues(t1)
  z2 = zValues(t2)
  
  out0 = outliers(z0)
  out1 = outliers(z1)
  out2 = outliers(z2)

In [1746]:
out0

,Index,_1,_2,_3,_4,target,_1_zscore,_2_zscore,_3_zscore,_4_zscore,target_zscore
15,15,5.7,4.4,1.5,0.4,0,1.988841,2.590242,0.221035,1.476136,NaN
24,24,4.8,3.4,1.9,0.2,0,-0.590348,-0.074616,2.547718,-0.440924,NaN
44,44,5.1,3.8,1.9,0.4,0,0.269382,0.991327,2.547718,1.476136,NaN


In [1747]:
out1

,Index,_1,_2,_3,_4,target,_1_zscore,_2_zscore,_3_zscore,_4_zscore,target_zscore


In [1748]:
out2

,Index,_1,_2,_3,_4,target,_1_zscore,_2_zscore,_3_zscore,_4_zscore,target_zscore
17,117,7.7,3.8,6.7,2.2,2,1.766513,2.58727,2.101225,0.639965,NaN
31,131,7.9,3.8,6.4,2.0,2,2.084231,2.58727,1.552125,-0.095627,NaN
